In [ ]:
import numpy as np
import os
import pandas as pd
import time
import xarray

In [ ]:
""" Read .nc file into pandas dataframe."""
#Runtime around 677 seconds.

tic = time.process_time()

nc_ibtracs = 'Allstorms.ibtracs_all.v03r10.nc'
path_to_nc = os.path.join('..','deconstruct_cyn',nc_ibtracs)

ibt_all_fields = xarray.open_dataset(path_to_nc).to_dataframe()
# name, storm_sn, time_record, landfall, 
# genesis_basin, season, 
# lat, lon, max_wind, min_pres, dist2land are the interesting variables

columns_we_want = ['storm_sn',
                   'name',
                   'source_time',
                   'landfall',
                   'genesis_basin',
                   'season',
                   'source_lat',
                   'source_lon',
                   'source_wind',
                   'source_pres',
                   'dist2land']

ibt = ibt_all_fields[columns_we_want]

renamed_columns = ['storm_sn',
                   'name',
                   'time_recorded',
                   'landfall',
                   'genesis_basin',
                   'season',
                   'lat',
                   'lon',
                   'max_wind',
                   'min_pres',
                   'dist2land']

columns = dict(zip(columns_we_want, renamed_columns))

toc = time.process_time()

elapsed = toc - tic
print(elapsed)
#ibt

In [ ]:
ibt_all_fields.loc[:]

In [ ]:
ibt.rename(index=str, columns = columns)

In [ ]:
ibt.rename(index=str, columns = columns)

In [ ]:
""" Import landmask."""
#Elapsed time ~10 seconds.
tic = time.process_time()

nc_landmask = 'ETOPO1_Ice_g_gmt4.nc' #Land mask from ETOPO1 Global Relief Model
path_to_nc2 = os.path.join('..','deconstruct_cyn',nc_landmask)

landmask_all_fields = xarray.open_dataset(path_to_nc2).to_dataframe()

"""
columns_we_want = ['storm_sn',
                   'name',
                   'source_time',
                   'landfall',
                   'genesis_basin',
                   'season',
                   'source_lat',
                   'source_lon',
                   'source_wind',
                   'source_pres',
                   'dist2land']

landmask = landmask_all_fields[columns_we_want]

columns = dict(zip(columns_we_want, renamed_columns))
"""

toc = time.process_time()

elapsed = toc - tic
print(elapsed)

In [ ]:
landmask_all_fields

In [ ]:
landmask_all_fields.index.names

In [ ]:
63867726/12861

In [ ]:
191 * 26